<a href="https://colab.research.google.com/github/iesous-kurios/powerBi/blob/master/PowerBI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [404]:
#@title
# import libraries which provide math functions to our data imports
import statistics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xlwt 
from xlwt import Workbook
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib
from matplotlib import style

sns.__version__

'0.10.0'

In [405]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#@title
# import weekly data from yearly FPS folder
df = pd.read_excel('/content/drive/My Drive/FPData/YTD Data/YTD_mar16.xlsx')
#df = pd.read_excel ('~/environments/fp_env/family_promise_data/data/ClientTrackExport (5).xlsx')

In [0]:
# get list of currently enrolled guests (guests that don't have an exit date)
currently_enrolled = df[df['3.11 Exit Date'].isnull()]

In [0]:
#currently_enrolled.head()

In [0]:
# create new list of those currently enrolled who haven't stayed with us for
# two weeks or more

from datetime import date, timedelta

current_date = date.today().isoformat()   
days_before = (date.today()-timedelta(days=30)).isoformat()

need_to_exit = currently_enrolled[currently_enrolled['Date of Last ES Stay (Beta)'] < days_before ]

In [0]:
exit_df = need_to_exit[['3.1 FirstName', '3.1 LastName', '5.8 Personal ID', '5.9 Household ID', '3.15 Relationship to HoH', 'CaseMembers', '3.10 Enroll Date', 'Date of Last ES Stay (Beta)']].copy()

In [0]:
#exit_df.head()

In [0]:
exit_df.to_excel('/content/drive/My Drive/FPData/YTD Data/YTDexit_df.xlsx')

In [413]:
households = df['5.9 Household ID'].unique()
len(households)

234

In [414]:
# print total number of families served (unduplicated)
unique_households = df['5.9 Household ID'].nunique()
unique_households

234

In [0]:
#@title
# replace zeros in data with "Nan" to calculate more accurate average bednights 
df['Count of Bed Nights (Housing Check-ins)'] = df['Count of Bed Nights (Housing Check-ins)'].replace(0, np.NaN)

In [416]:
avg_bn = df['Count of Bed Nights (Housing Check-ins)'].mean()
avg_bn = round(avg_bn, 2)
avg_bn

34.65

In [0]:
#@title

exit_reasons = ['Rental by client with RRH or equivalent subsidy', 
                'Rental by client, no ongoing housing subsidy', 
                'Staying or living with family, permanent tenure', 
                'Rental by client, other ongoing housing subsidy',
                'Permanent housing (other than RRH) for formerly homeless persons', 
                'Staying or living with friends, permanent tenure', 
                'Owned by client, with ongoing housing subsidy', 
                'Staying or living with friends, permanent tenure', 
                'Rental by client, VASH housing Subsidy'
               ]

temp_reasons = ['Staying or living with family, temporary tenure (e.g., room, apartment or house)', 
               'Staying or living with friends, temporary tenure (e.g., room, apartment or house)', 
                
               ]

In [0]:
# drop rows with no exit destination (current guests at time of report)
df = df.dropna(subset=['3.12 Exit Destination'])

In [0]:
 # create target column (multiple types of exits to perm)
df['perm_leaver'] = df['3.12 Exit Destination'].isin(exit_reasons)

In [0]:
#df.head()

In [0]:
perm_leaver = df[df['perm_leaver'] == True]

In [0]:
total_perm = perm_leaver.perm_leaver.value_counts().sum()

In [423]:
total_perm

196

In [0]:
non_perm = df[df['perm_leaver'] == False]

In [0]:
total_non_perm = non_perm.perm_leaver.value_counts().sum()

In [426]:
total_non_perm

408

In [0]:
perm_percent = total_perm / total_non_perm

In [428]:
perm_percent = round(perm_percent, 2)*100
perm_percent

48.0

In [429]:
df['3.12 Exit Destination'].value_counts()

Client refused                                                                                                                   171
Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter             123
Rental by client with RRH or equivalent subsidy                                                                                   81
Rental by client, no ongoing housing subsidy                                                                                      73
Transitional Housing for homeless persons (including homeless youth)                                                              38
No exit interview completed                                                                                                       23
Rental by client, other ongoing housing subsidy                                                                                   21
Staying or living with family, permanent tenure                      

In [0]:
unknowns = ['Client refused', 'No exit interview completed']

In [0]:
unknown_df = df[df['3.12 Exit Destination'].isin(unknowns)]

In [0]:
unknown_exits = unknown_df['3.12 Exit Destination'].value_counts().sum()

In [433]:
unknown_exits

194

In [434]:
unknown_percent = unknown_exits / total_non_perm
unknown_percent = round(unknown_percent, 2) *100
unknown_percent

48.0

In [435]:
exits = df['3.12 Exit Destination'].value_counts().sum()
exits

604

In [0]:
exits = df['3.12 Exit Destination'].value_counts()

In [437]:
# take sum of ES and Trans exits and divide by total exits, then multiply by 100 to get percentage
esExits = exits['Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter'] 
#transExits = ((exits['Transitional Housing for homeless persons (including homeless youth)'] / exits.sum()) * 100)
# round the answer to nearest hundreth place
esExits = (round(esExits, 2))
#transExits = (round(transExits, 2))
esExits
#print('Exits to Transitional',transExits,'%')

123

In [438]:
tempExits = df['3.12 Exit Destination'].isin(temp_reasons).sum()

tempExits

35

In [439]:
transitional = df['3.12 Exit Destination'] == 'Transitional Housing for homeless persons (including homeless youth)'
transitional = transitional.sum()
transitional

38

In [440]:
exits.sum()

604

In [0]:
total_guests = total_non_perm + total_perm

In [442]:
trans_percent = (transitional / exits.sum())*100
trans_percent = round(trans_percent, 2)
trans_percent

6.29

In [443]:
zips_master

,V5 Zip,V5 City,V5 State,county_name
0,601,Adjuntas,Puerto Rico,Adjuntas
1,602,Aguada,Puerto Rico,Aguada
2,603,Aguadilla,Puerto Rico,Aguadilla
3,606,Maricao,Puerto Rico,Maricao
4,610,Anasco,Puerto Rico,Añasco
...,...,...,...,...
33095,99925,Klawock,Alaska,Prince of Wales-Hyder
33096,99926,Metlakatla,Alaska,Prince of Wales-Hyder
33097,99927,Point Baker,Alaska,Prince of Wales-Hyder
33098,99929,Wrangell,Alaska,Wrangell


In [467]:
# export zips into an excel sheet for Power BI visual
zips_master = pd.read_excel('/content/drive/My Drive/FPData/uszips.xlsx')



# Import pandas library 
import pandas as pd 
  
# initialize list of lists 
zips_bi = df['V5 Zip'].value_counts().to_frame('Total').reset_index(level=0, inplace=False)
zips_bi = zips_bi.rename(columns={"index": "V5 Zip"})
zips_bi = zips_bi.merge(zips_master,on='V5 Zip')
zips_bi

,V5 Zip,Total,V5 City,V5 State,county_name
0,99201.0,36,Spokane,Washington,Spokane
1,99207.0,28,Spokane,Washington,Spokane
2,99205.0,23,Spokane,Washington,Spokane
3,99206.0,21,Spokane Valley,Washington,Spokane
4,99208.0,20,Spokane,Washington,Spokane
...,...,...,...,...,...
59,72653.0,1,Mountain Home,Arkansas,Baxter
60,99217.0,1,Spokane,Washington,Spokane
61,59401.0,1,Great Falls,Montana,Cascade
62,98208.0,1,Everett,Washington,Snohomish


In [470]:
type(zips_bi)

pandas.core.frame.DataFrame

In [0]:
zips_bi.to_excel('/content/drive/My Drive/FPData/YTD Data/weeklyYTDzips.xlsx', header=True, index=False)

In [0]:
other_reasons = ['Place not meant for habitation (e.g., a vehicle, an abandoned building, bus/train/subway station/airport or anywhere outside)',
                 'Hotel or Motel paid for without Emergency Shelter Voucher', 'Other', 'Substance Abuse Treatment or Detox Center',
                 'Foster Care Home or Foster Care Group Home', 'Psychiatric Hospital or Other Psychiatric Facility']

In [0]:
other = df['3.12 Exit Destination'].isin(other_reasons)

In [450]:
other.sum()
other = other.sum()
other

18

In [451]:
df['4.04.7 Private - Employer']

0       No
1       No
2       No
3       No
4       No
      ... 
690    NaN
706     No
707     No
708     No
709     No
Name: 4.04.7 Private - Employer, Length: 604, dtype: object

In [452]:
df['3.12 Exit Destination'].value_counts()

Client refused                                                                                                                   171
Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter             123
Rental by client with RRH or equivalent subsidy                                                                                   81
Rental by client, no ongoing housing subsidy                                                                                      73
Transitional Housing for homeless persons (including homeless youth)                                                              38
No exit interview completed                                                                                                       23
Rental by client, other ongoing housing subsidy                                                                                   21
Staying or living with family, permanent tenure                      

In [453]:
print(total_guests, 'total guests served YTD')
print(total_non_perm, 'total non perm leavers')
print(total_perm, 'total perm leavers')
print(esExits, 'total ES leavers')
print(tempExits, 'total temp leavers')
print(transitional, 'total trans leavers')
print(other, 'total leavers to "other"')
print(unknown_exits, 'total unknown leavers')
print('sum of all groups to make sure it matches total served:',total_perm+esExits+tempExits+transitional+other+unknown_exits)

604 total guests served YTD
408 total non perm leavers
196 total perm leavers
123 total ES leavers
35 total temp leavers
38 total trans leavers
18 total leavers to "other"
194 total unknown leavers
sum of all groups to make sure it matches total served: 604


In [0]:
assert((total_perm+esExits+tempExits+transitional+other+unknown_exits) == (total_non_perm+total_perm))

In [0]:
perm_bi = '{} %'.format(perm_percent)
es_bi = '{} %'.format(esExits)
temp_bi = '{} %'.format(tempExits)
trans_bi = '{} %'.format(trans_percent)
unknown_bi = '{} %'.format(unknown_percent)

In [0]:
power_bi = pd.DataFrame({'Permanent Leavers': total_perm, 'Emergency Shelter Leavers': esExits, 
                         'Temporary Leavers': tempExits, 'Transitional Leavers': transitional, 
                         'Unknown Leavers': unknown_exits, 'Other Leavers': other, 'Total': total_guests }, index=[0])


In [0]:
# Import pandas library 
import pandas as pd 
  
# initialize list of lists 
data = [['Permanent Leavers', total_perm], ['Emergency Shelter Leavers', esExits], 
        ['Temporary Leavers', tempExits], ['Transitional Leavers', transitional],
        ['Unknown Leavers', unknown_exits], ['Other Leavers', other]] 
  


# Create the pandas DataFrame 
power_bi = pd.DataFrame(data, columns = ['Exit Destination', 'Total']) 
  
# print dataframe. 

In [458]:
power_bi

,Exit Destination,Total
0,Permanent Leavers,196
1,Emergency Shelter Leavers,123
2,Temporary Leavers,35
3,Transitional Leavers,38
4,Unknown Leavers,194
5,Other Leavers,18


In [459]:
power_bi.Total.sum()

604

In [0]:
power_bi.to_excel('/content/drive/My Drive/FPData/YTD Data/YTDweeklyDataNew.xlsx')